In [1]:
import pandas as pd
import re
import openpyxl
import nltk
from nltk.corpus import stopwords

En primer lugar importamos los excel correspondientes a ambos datasets (SPAM y NO SPAM)

In [2]:
data = pd.read_excel('../data_spam/SPAM_JMA.xlsx')
data_main = pd.read_excel('../data_spam/2024-TODOS.xlsx')

In [3]:
data.head()

,eml_body,eml_from
0,\t_x000d_\n\tSi vous ne parvenez pas à lire ce...,info@news.seleccionfiguras.com
1,\tAlgunos contactos que recibieron este mensaj...,hanson@dyparking.co.kr
2,Ce message vous est adressé par Manageo pour l...,m.michel@mp.aconclue-pro.com
3,"Bonjour M. RENASSIA, _x000d_\n_x000d_\n_x000d_...",sandra-de-rgpd@veml02.com
4,<https://nl.cepartsoftware.com/MCP53133623730...,staff.news@cepartscatalog.it


In [4]:
data_main.head()

,CodigoTarea,IDEmail,De,Cuerpo,FechaCreacion,Categoria,Idioma
0,1176970,1672446,ecommerce.spareparts@hidral.com,_x000d_\n_x000d_\n _x000d_\n_x000d_\nDe: eCom...,2024-01-09,OREP,E
1,1177144,1672695,a.gallardo@hidral.fr,Buenas:_x000d_\nHemos recibido este mail de un...,2024-01-09,DOC,E
2,1177737,1673551,reisvila@hosteriasdetarragona.cat,"Buenos días Sr. González,_x000d_\n_x000d_\nEn ...",2024-01-10,C,E
3,1177864,1673745,sulift@sapo.pt,"Boa Tarde, _x000d_\n_x000d_\n _x000d_\n_x000d_...",2024-01-10,OREP,E
4,1177864,1673881,sulift@sapo.pt,"Boa tarde, _x000d_\n_x000d_\n _x000d_\n_x000d_...",2024-01-10,OREP,E


Creamos una función para limpiar los datos de cada uno de los contenidos del cuerpo

In [5]:
# Asegúrate de descargar las stopwords
nltk.download('stopwords')

# Función para limpiar el texto
def clean_text(text):
    text = str(text)
    # Eliminar caracteres no deseados (como xd)
    text = re.sub(r'\bxd\b', '', text)
    
    # Eliminar caracteres no deseados (como x000d)
    text = re.sub(r'\bx000d\b', '', text)
    
    # Eliminar URLs y correos electrónicos
    text = re.sub(r'http\S+|www\S+|https\S+|mailto:\S+', '', text)
    
    # Mantener solo palabras en español (puedes ajustar esto según tus necesidades)
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos y espacios extra
    text = re.sub(r'[^a-záéíóúñü\s]', '', text)
    
    # Dividir el texto en palabras
    words = text.split()
    
    # Filtrar stopwords
    spanish_stopwords = set(stopwords.words('spanish'))
    filtered_words = [word for word in words if word not in spanish_stopwords]
    
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /home/jose-
[nltk_data]     manuel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Realizamos los cambios pertinentes para el Dataframe de los **SPAM**

In [6]:
data['eml_body'] = data['eml_body'].apply(clean_text) 
data['Categoria'] = 'SPAM'
data['Cuerpo'] = data['eml_body'] + data['eml_from']
data = data[['Cuerpo', 'Categoria']]
data.head()

,Cuerpo,Categoria
0,xd si vous ne parvenez pas lire ce message vis...,SPAM
1,contactos recibieron mensaje suelen recibir co...,SPAM
2,ce message vous est adressé par manageo pour c...,SPAM
3,bonjour m renassia xd xd xd tesvous sre garant...,SPAM
4,xd xd xd hábito hace monje layout hace catálog...,SPAM


Hacemos lo mismo para los **NO SPAM**. En este caso tenemos que realizar también un *group by* para ignorar los correos de una misma tarea que no sea el primero, para el correcto funcionamiento del modelo posteriormente. 

In [7]:
data_main = data_main.groupby(by='CodigoTarea').agg({
        'IDEmail': 'first',
        'De': 'first',
        'Cuerpo': 'first',
        'FechaCreacion': 'first',
        'Categoria': 'first',
        'Idioma': 'first'
    }).reset_index()
data_main['Cuerpo'] = data_main['Cuerpo'].apply(clean_text)
data_main['Cuerpo'] = data_main['Cuerpo'] + data_main['De']
data_main = data_main[['Cuerpo', 'Categoria']]
data_main.head()

,Cuerpo,Categoria
0,buen día felices fiestasxd xd hace año hizo pe...,SAT
1,buenos díasxd xd xd xd enrique millán antonio ...,SEG
2,good morningxd xd xd xd please find below our ...,SEG
3,hello xd xd is it possible to share the quote ...,OREP
4,buenos díasxd xd xd xd podéis indicar favor re...,SEG


Por último tenemos que unir ambos para meterlos al modelo

In [8]:
combined_data = pd.concat([data, data_main])

In [11]:
combined_data = combined_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
combined_data.head()

,Cuerpo,Categoria
0,dear colleauguesxd xd xd xd kindly find below ...,OMOD
1,papeles lelgado hoy fabrica menos martes xdjes...,SEG
2,xd xd xd xd xd xd xd xd xd xd xd xd xd xd xd x...,SPAM
3,xd ver online xd xd xd xd xd especialistas gam...,SPAM
4,hi oscar xd xd i need the programming details ...,DOC


In [13]:
combined_data.isna().sum()

Cuerpo       4
Categoria    0
dtype: int64